In [1]:
import random

In [9]:
#-------------------------------------------------------------
# Public parameter generator input = degree k
#-------------------------------------------------------------
def parameter_gen(k):
    flag = True
    while flag :
        N = next_prime(randrange(2**k, 2**(k+1)))
        p = next_prime(randrange(2**k, 2**(k+1)))
        d = randrange(1, N)
        q = next_prime(randrange((6*d + 1)*p, (6*d + 1)*p*2**5))
        if gcd(p,q) == 1 :
            if gcd(N,q) == 1 :
                if q > (6*d + 1) * p :
                    flag = False
    return (N,p,q,d)
#-------------------------------------------------------------
k = 7
(N,p,q,d) = parameter_gen(k)
print ("(N,p,q,d) = " + str((N,p,q,d)))

(N,p,q,d) = (139, 149, 2131771, 135)


In [10]:
#-------------------------------------------------------------
def tri_poly(d_1, d_2) :
    s = [1 for j in range(d_1 - 1)]
    s = s + [-1 for j in range(d_2)]
    s = s + [0 for j in range(N - d_2 - d_1)]
    random.shuffle(s)
    s.append(1)
    return R(s)
#-------------------------------------------------------------
def fq_inv(f) :
    return xgcd(id, f)[2]%q
#-------------------------------------------------------------
def fp_inv(f) :
    return xgcd(id, f)[2]%p
#-------------------------------------------------------------
def find_h(f, g) :
    return (f * g % id) % q

In [5]:
#-------------------------------------------------------------
# Key generated by Alice
#-------------------------------------------------------------
(N,p,q,d) = (7,3,41,2)
#-------------------------------------------------------------
# Generate R[x]/(x^N-1)
#-------------------------------------------------------------
R.<x> = PolynomialRing(ZZ)
id = x**N-1
#-------------------------------------------------------------
f = tri_poly(d+1, d)
g = tri_poly(d, d)
F_q = fq_inv(f)
F_p = fp_inv(f)
h = find_h(F_q, g)
print "Alice's private key (f,F_p): "
show((f,F_p))
print "Alice's public key (h): "
show(h)
#latex(f)
#latex(F_p)
#latex(h)

Alice's private key (f,F_p): 


(x^6 + x^5 - x^4 + x - 1, 2*x^5 + x^2 + 2)

Alice's public key (h): 


-5*x^6 + 39*x^5 - 29*x^4 - 8*x^3 - 12*x^2 + 12*x + 3

In [11]:
#-------------------------------------------------------------
# Encryption Function
#-------------------------------------------------------------
def encrypt(m, h, r) :
    return (((p*r*h)%id)%q+m)%q
#-------------------------------------------------------------
# Public parameter
#-------------------------------------------------------------
(N,p,q,d) = (7,3,41,2)
#-------------------------------------------------------------
# Generate R[x]/(x^N-1)
#-------------------------------------------------------------
R.<x> = PolynomialRing(ZZ)
id = x**N-1
#-------------------------------------------------------------
# Key generated by Alice
#-------------------------------------------------------------
f = x^6-x^4+x^3+x^2-1
g = x^6+x^4-x^2-x
F_q = fq_inv(f)
F_p = fp_inv(f)
h = find_h(F_q, g)
print "Alice's private key (f,F_p): "
show((f,F_p))
#latex(f)
#latex(F_p)
print "Alice's public key (h): "
show(h)
#latex(h)
print "Alice's public key same as example(h): "
show(20*40*h%q)
#latex(20*40*h%q)
#-------------------------------------------------------------
# Bob encrypt message
#-------------------------------------------------------------
m = -x^5 + x^3 + x^2 - x + 1
#-------------------------------------------------------------
# Ephimeral key
#-------------------------------------------------------------
r= x^6 - x^5 + x-1
#-------------------------------------------------------------
e = encrypt(m, h, r)
print "Plant text : "
show(m)
#latex(m)
print "Cypher text with first public key: "
show(e)
#latex(e)
e = encrypt(m, 20*40*h%q, r)
print "Cypher text with public key same as example: "
show(e)
#latex(e)

Alice's private key (f,F_p): 


(x^6 - x^4 + x^3 + x^2 - 1, x^6 + 2*x^5 + x^3 - 2*x^2 + x + 1)

Alice's public key (h): 


40*x^6 - 2*x^5 + 4*x^4 + 35*x^3 + 16*x^2 + 11*x - 22

Alice's public key same as example(h): 


...

-x^5 + x^3 + x^2 - x + 1

Cypher text with first public key: 


21*x^6 + 39*x^5 - 33*x^4 - 38*x^3 + 38*x^2 + 7*x + 8

Cypher text with public key same as example: 


31*x^6 + 19*x^5 + 4*x^4 + 2*x^3 + 40*x^2 + 3*x + 25

In [12]:
#-------------------------------------------------------------
# Center lift function
#-------------------------------------------------------------
def center_lift(a, t) :
    lst_a = list(a)
    for i in range(len(lst_a)) :
        if lst_a[i] <= -t/2 :
            lst_a[i] += t
        elif lst_a[i] > t/2 :
            lst_a[i] -= t
    return R(lst_a)
#-------------------------------------------------------------
# Decrypt recieved cypher text e(x)
#-------------------------------------------------------------
def decrypt(f, F_p, e) :
    a = ((f * e) % id) % q
    a = center_lift(a, q)
    mhat = (F_p * a % id) % p
    mhat = center_lift(mhat, p)
    return mhat
#-------------------------------------------------------------

In [8]:
#--------------------------------------------------------
# Alice decrypt cypher text
#--------------------------------------------------------
# First cypher text e(x)
#--------------------------------------------------------
e = 21*x^6 + 39*x^5 - 33*x^4 - 38*x^3 + 38*x^2 + 7*x + 8
mhat_lift = decrypt(f, F_p, e)
print "Recovery message (for ﬁrst cipher): "
show(mhat_lift)
#--------------------------------------------------------
# Second cypher text e(x)
#--------------------------------------------------------
e = 31*x^6 + 19*x^5 + 4*x^4 + 2*x^3 + 40*x^2 + 3*x + 25
mhat_lift = decrypt(f, F_p, e)
print "Recovery message (for second cipher): "
show(mhat_lift)
#--------------------------------------------------------

Recovery message (for ﬁrst cipher): 


-x^5 + x^3 + x^2 - x + 1

Recovery message (for second cipher): 


-x^5 + x^3 + x^2 - x + 1